In [1]:
import re
import csv


sample_re = re.compile(r'Generating sample (\d+) \.\.\.')
entropy_re = re.compile(r'Entropy: ([\d.]+)')
timestep_re = re.compile(r'TIMESTEP \[(\d+)/(\d+)\].*current PLDDT: ([\d.]+)')

# List to hold extracted data
extracted_data = []

In [28]:
input_file_path = './secondary_structure_wm.log'

In [29]:
with open(input_file_path, 'r') as file:
    sample_number = None
    entropy = None
    for line in file:
        line = line.strip()
        if not line:  # Skip empty lines
            continue

        # Match each line against the regular expressions
        sample_match = sample_re.search(line)
        entropy_match = entropy_re.search(line)
        timestep_match = timestep_re.search(line)
    
        if sample_match:
            sample_number = sample_match.group(1)
        elif entropy_match:
            entropy = entropy_match.group(1)
        elif timestep_match:
            current_iteration, total_iterations, current_plddt = timestep_match.groups()
            modified_iteration = int(total_iterations) + 1 - int(current_iteration)
            extracted_data.append({
                'sample': sample_number,
                'iteration': int(modified_iteration),
                'entropy': float(entropy),
                'pLDDT': float(current_plddt),
            })

In [30]:
import pandas as pd

# Convert extracted data into a pandas DataFrame
df = pd.DataFrame(extracted_data)

# Display the DataFrame to ensure it's correctly formatted
df.tail()

,sample,iteration,entropy,pLDDT
7495,000049,21,0.737793,0.7305
7496,000049,22,0.704590,0.6548
7497,000049,23,0.883301,0.6831
7498,000049,24,0.975586,0.6963
7499,000049,25,1.250000,0.7368


In [31]:
df.to_csv('secondary_structure_wm.csv')